# Project 2
## Lawrence Liu Jackie McGinley

# Background
#### A large company named Canterra, employs, at any given point of time, around 4000 employees. However, every year, around 15% of its employees leave the company and need to be replaced with the talent pool available in the job market. The management believes that this level of attrition (employees leaving, either on their own or because they got fired) is bad for the company, because of the following reasons:
- The former employees’ projects get delayed, which makes it difficult to meet timelines, resulting in a reputation loss among consumers and partners
- A sizable department has to be maintained, for the purposes of recruiting new talent
- More often than not, the new employees have to be trained for the job and/or given time to acclimatize themselves to the company
#### The management hypothesizes that higher job satisfaction and a higher number of total working years will reduce employee attrition. Additionally, the marketing management was interested to know if demographic variables such as gender, education and age affect employee attrition. Hence, the management has contracted you as a consultant to understand whether these two factors they should focus on, in order to curb attrition. In other words, they want to know if changes in their internal and external recruitment strategies would help retain employees.



In [5]:
### Step 0: Setup ----
# Load any libraries used for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import statsmodels.api as sm

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    roc_curve,
    roc_auc_score
)


In [6]:
# -------------------------------------------------------------------
# Step 1: Pre-processing
# -------------------------------------------------------------------

print("Do we have missing values? Look at 'Non-Null Count'")
df = pd.read_excel('Employee_Data_Project.xlsx')
df.shape
print(df.info())


Do we have missing values? Look at 'Non-Null Count'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EmployeeID               4410 non-null   int64  
 6   Gender                   4410 non-null   object 
 7   JobLevel                 4410 non-null   int64  
 8   MaritalStatus            4410 non-null   object 
 9   Income                   4410 non-null   int64  
 10  NumCompaniesWorked       4391 non-null   float64
 11  StandardHours            4410 non-null   int64  
 12  TotalWorkingYears        4401 non-null   float64
 13  TrainingTimesLastYear    4

#### There are missing values in the NumCompaniesWorked , TotalWorkingYears , EnvironmentSatisfaction , JobSatisfaction Columns

In [7]:
# -------------------------------------------------------------------
# Step 1: Pre-processing (check rows with nulls and visualize those rows)
# -------------------------------------------------------------------

# first step is to get the index of where the nulls are and take a look at only those rows
null_rows = df[df.isnull().any(axis=1)].index
# use the index from null rows to view those rows
df.loc[null_rows]
## confirmed that there are truly NaNs (73 total rows have NaNs)

,Age,Attrition,BusinessTravel,DistanceFromHome,Education,EmployeeID,Gender,JobLevel,MaritalStatus,Income,NumCompaniesWorked,StandardHours,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction
11,36,No,Travel_Rarely,28,1,12,Male,1,Married,33770,0.0,8,16.0,2,15,11,NaN,4.0
23,42,No,Travel_Rarely,4,4,24,Male,1,Married,89260,1.0,8,NaN,4,20,6,2.0,3.0
40,36,No,Travel_Frequently,8,3,41,Female,3,Married,69620,4.0,8,4.0,2,1,0,3.0,NaN
111,31,No,Travel_Rarely,1,3,112,Male,4,Single,28670,0.0,8,3.0,5,2,2,NaN,2.0
115,27,No,Travel_Rarely,2,3,116,Male,1,Divorced,23670,NaN,8,5.0,2,5,4,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,36,No,Travel_Rarely,2,3,4227,Male,2,Married,43200,NaN,8,5.0,2,1,0,2.0,4.0
4332,31,No,Travel_Rarely,2,5,4333,Male,2,Married,27280,8.0,8,7.0,3,4,2,NaN,4.0
4345,43,No,Non-Travel,6,2,4346,Male,1,Divorced,20280,4.0,8,7.0,2,5,2,4.0,NaN
4395,40,No,Travel_Rarely,2,3,4396,Male,1,Divorced,27180,NaN,8,9.0,4,9,7,1.0,4.0


In [8]:
# -------------------------------------------------------------------
# Step 1: Pre-processing (remove NaNs)
# -------------------------------------------------------------------
df = df.dropna()
df.shape
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4337 entries, 0 to 4408
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4337 non-null   int64  
 1   Attrition                4337 non-null   object 
 2   BusinessTravel           4337 non-null   object 
 3   DistanceFromHome         4337 non-null   int64  
 4   Education                4337 non-null   int64  
 5   EmployeeID               4337 non-null   int64  
 6   Gender                   4337 non-null   object 
 7   JobLevel                 4337 non-null   int64  
 8   MaritalStatus            4337 non-null   object 
 9   Income                   4337 non-null   int64  
 10  NumCompaniesWorked       4337 non-null   float64
 11  StandardHours            4337 non-null   int64  
 12  TotalWorkingYears        4337 non-null   float64
 13  TrainingTimesLastYear    4337 non-null   int64  
 14  YearsAtCompany           4337

#### convert the object datatype to integers

In [50]:
df = pd.read_excel('Employee_Data_Project.xlsx')
df['Attrition'] = pd.Categorical(df['Attrition'])
df = pd.get_dummies(df, columns = ['Attrition'], drop_first = True)
df['Gender'] = pd.Categorical(df['Gender'])
df = pd.get_dummies(df, columns= ['Gender'] , drop_first = True )
for col in df.select_dtypes(['bool']).columns:
    df[col] = df[col].astype(int)


In [52]:
df.columns
df['Gender_Male'].dtype

dtype('int64')

In [54]:
# -------------------------------------------------------------------
# Step 2: Train Test Split
# -------------------------------------------------------------------
X = df.drop(['Attrition_Yes','EmployeeID'], axis =1)
y = df['Attrition_Yes']

X_train , X_test, y_train, y_test = train_test_split(
    X, y, test_size  = 0.3, random_state = 42 , stratify= y
)

X_train.info

<bound method DataFrame.info of       Age     BusinessTravel  DistanceFromHome  Education  JobLevel  \
2685   33  Travel_Frequently                10          2         2   
3631   39      Travel_Rarely                26          1         2   
1365   45      Travel_Rarely                 1          4         1   
3623   30      Travel_Rarely                 9          1         4   
4390   32      Travel_Rarely                13          4         3   
...   ...                ...               ...        ...       ...   
3369   36      Travel_Rarely                 9          3         2   
3858   49      Travel_Rarely                 4          2         2   
743    37      Travel_Rarely                20          5         2   
808    33      Travel_Rarely                 7          2         2   
1125   36         Non-Travel                 1          3         2   

     MaritalStatus  Income  NumCompaniesWorked  StandardHours  \
2685       Married   34200                 1.0    

In [55]:
## placeholder will use for actual charts later 


base =alt.Chart(df).mark_point(opacity=0.4).encode(
    alt.X('log_gdp' , title= 'Log GDP').axis(ticks=False).scale(domain = (4,12)),
    alt.Y('lifeExp' , title = 'Life Expectancy Value').axis(ticks=False).scale(domain = (10,90)),
    color = 'continent:N',
    tooltip = ['continent' , 'country' , 'year']

).properties(
    title = 'Scatterplot of Life Expectancy Vs. Log GDP',
    width = 1000,
    height = 800
)

trendline = base.transform_regression('log_gdp' , 'lifeExp' , method = 'linear' , groupby= ['continent']).mark_line( size=2)

chart = (base + trendline).configure_axis(grid=False)
chart.show()

ValueError: Unable to determine data type for the field "continent"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

alt.LayerChart(...)

# Question 1

#### 1: Explain why logistic regression is an appropriate modeling technique for predicting employee attrition in this dataset compared to classical regression methods. 

#### 2: Provide a specific rationale for utilizing logistic regression in predicting attrition concerning the age of employees. 



# Question 2

#### Model Comparison - Choose four variables from the list provided (e.g., `Age`, `Gender`, `JobSatisfaction`, `Income`) and use them to create logistic regression models with the following combinations:
- Model 1: A one-variable model with `Age`.
- Model 2: A two-variable model with `Age` and `Gender`
- Model 3: A three-variable model with `Age`, `Gender`, and `JobSatisfaction`.
- Model 4: An interaction model involving `Age`, `Gender`, `JobSatisfaction`, `Income`, and `Gender:Income`. 

In [56]:
## model 1: One Variable model with Age 

# define 5-fold stratified cross validation: 

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

cv = KFold(
    n_splits = 5, 
    shuffle = True,
    random_state = 42
)

log_reg = LogisticRegression(
    max_iter = 2500
)

model_1_scores = cross_val_score(
    log_reg, 
    X_train[['Age']],
    y_train,
    cv= cv, 
    scoring = "accuracy"
)
print("CV accuracy scores:", model_1_scores)
print("Mean accuracy:", model_1_scores.mean())



CV accuracy scores: [0.84142395 0.85436893 0.80226904 0.84602917 0.84927066]
Mean accuracy: 0.838672352388895


#### Need to convert gender from object to category (use one hot encoding / dummy variables) 

In [57]:
## model 2: 

model_2_scores = cross_val_score( 
    log_reg, 
    X_train[['Age','Gender_Male']],
    y_train, 
    cv=cv, 
    scoring = 'accuracy'
)
print("CV accuracy scores:", model_2_scores)
print("Mean accuracy:", model_2_scores.mean())

CV accuracy scores: [0.84142395 0.85436893 0.80226904 0.84602917 0.84927066]
Mean accuracy: 0.838672352388895


In [58]:
model_3_score = cross_val_score( 
    log_reg, 
    X_train[['Age' , 'Gender_Male' , 'JobSatisfaction']],
    y_train, 
    cv=cv,
    scoring = "accuracy"
)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\liula\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\liula\anaconda3\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\liula\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1222, in fit
    X, y = validate_data(
           ~~~~~~~~~~~~~^
        self,
        ^^^^^
    ...<5 lines>...
        accept_large_sparse=solver not in ["liblinear", "sag", "saga"],
        ^^^

#### test model 1 precision and recall 


In [ ]:
model_1 = sm.Logit(y_train, sm.add_constant(X_train['Age'])).fit()
# get a summary
print("\n=== model 1 summary ===")
print(model_1.summary())

Optimization terminated successfully.
         Current function value: 0.428362
         Iterations 6

=== model 1 summary ===
                           Logit Regression Results                           
Dep. Variable:          Attrition_Yes   No. Observations:                 3035
Model:                          Logit   Df Residuals:                     3033
Method:                           MLE   Df Model:                            1
Date:                Mon, 24 Nov 2025   Pseudo R-squ.:                 0.02981
Time:                        10:57:02   Log-Likelihood:                -1300.1
converged:                       True   LL-Null:                       -1340.0
Covariance Type:            nonrobust   LLR p-value:                 3.971e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1788      0.212      0.844      0.399      -0.237       0.594
Age 

In [38]:
# This function takes your fitted model as input
# It iterates across all variables
# And it produces a pandas data frame of VIFs for each variable

from statsmodels.stats.outliers_influence import variance_inflation_factor

def vif_table(model):
    X = model.model.exog
    names = model.model.exog_names
    vif_values = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
    return pd.DataFrame({"variable": names, "VIF": vif_values})


vif_model_1 = vif_table(model_1)

print("\nVIF (f1):")
print(vif_model_1)



VIF (f1):
  variable        VIF
0    const  17.263461
1      Age   1.000000


In [ ]:
# ROC/AUC/Precision/Recall on original (unbalanced) TRAIN set, using probabilities

# Predicted probabilities on training data
train_probs = model_1.predict(sm.add_constant(X_train[['Age']]))

# Calculate precision and recall (threshold 0.5)
train_pred_class = (train_probs >= 0.5).astype(int)

cm_train = confusion_matrix(y_train, train_pred_class)  # y_train directly
precision_train = precision_score(y_train, train_pred_class)  # y_train directly
recall_train = recall_score(y_train, train_pred_class)  # y_train directly

print("\nIn-sample confusion matrix (threshold = 0.5):")
print(pd.DataFrame(
    cm_train,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
))
print(f"\nPrecision (train): {precision_train:.3f}")
print(f"Recall (train):    {recall_train:.3f}")


In-sample confusion matrix (threshold = 0.5):
          Pred 0  Pred 1
Actual 0    2546       0
Actual 1     489       0

Precision (train): 0.000
Recall (train):    0.000


c:\Users\liula\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [60]:
model_2 = sm.Logit(y_train, sm.add_constant(X_train[['Age' , 'Gender_Male']])).fit()
# get a summary
print("\n=== model 1 summary ===")
print(model_2.summary())


vif_model_2 = vif_table(model_2)

print("\nVIF (f1):")
print(vif_model_2)


# ROC/AUC/Precision/Recall on original (unbalanced) TRAIN set, using probabilities

# Predicted probabilities on training data
train_probs2 = model_2.predict(sm.add_constant(X_train[['Age' , 'Gender_Male']]))

# Calculate precision and recall (threshold 0.5)
train_pred_class2 = (train_probs2 >= 0.5).astype(int)

cm_train2 = confusion_matrix(y_train, train_pred_class2)  # y_train directly
precision_train2 = precision_score(y_train, train_pred_class2)  # y_train directly
recall_train2 = recall_score(y_train, train_pred_class2)  # y_train directly

print("\nIn-sample confusion matrix (threshold = 0.5):")
print(pd.DataFrame(
    cm_train2,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
))
print(f"\nPrecision (train): {precision_train2:.3f}")
print(f"Recall (train):    {recall_train2:.3f}")

Optimization terminated successfully.
         Current function value: 0.424663
         Iterations 6

=== model 1 summary ===
                           Logit Regression Results                           
Dep. Variable:          Attrition_Yes   No. Observations:                 3087
Model:                          Logit   Df Residuals:                     3084
Method:                           MLE   Df Model:                            2
Date:                Mon, 24 Nov 2025   Pseudo R-squ.:                 0.03891
Time:                        11:27:13   Log-Likelihood:                -1310.9
converged:                       True   LL-Null:                       -1364.0
Covariance Type:            nonrobust   LLR p-value:                 8.931e-24
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3929      0.226      1.742      0.082      -0.049       0.835
A

c:\Users\liula\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
